In [119]:
import pandas as pd
import numpy as np
from numpy import newaxis
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier 

In [106]:
def match_data_to_labels(train, labels):
    columns_names=list(train.columns) + list(labels.columns)
    df = pd.DataFrame(np.zeros([labels.shape[0], train.shape[1]+1]), columns=columns_names, index=labels.index)
    for _id, seq in df.iterrows():
        for item in columns_names[:-1]:
            if _id in train.index:     
                seq[item] = train.at[_id, item]  
            else:
                continue
        seq["label"] = labels.at[_id,"label"]
    print(df.shape, )
    return df

#Creating training data
def create_training_data(train, test_size=0.25):
    x_train = train[features]
    y_train = train["label"]
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,random_state=10, test_size=test_size)
    x_train = x_train.to_numpy().reshape(x_train.shape[0],x_train.shape[1],1)
    x_test = x_test.to_numpy().reshape(x_test.shape[0],x_test.shape[1],1)
    onehot_encoder = OneHotEncoder(sparse=False)
    y_train = onehot_encoder.fit_transform(y_train.to_numpy().reshape(y_train.shape[0],1))
    y_test = onehot_encoder.fit_transform(y_test.to_numpy().reshape(y_test.shape[0],1))
    return x_train, x_test, y_train, y_test

In [137]:

# Structure train and test data
class_names = ["Pedestrian", "Bicycle", "Car"]
n_classes = len(class_names)

raw_data = pd.read_csv("training_data/features.csv").drop_duplicates(subset=["id"],keep="first").set_index("id")
labels = pd.read_csv("sequence_labels.csv").drop_duplicates(subset=["id"],keep="first").set_index("id")
data = match_data_to_labels(raw_data, labels)
features = data.columns.values[:-1]

x_train, x_test, y_train, y_test = create_training_data(data, test_size=0.25)

In [146]:

def create_model(optimizer="adam", activation="relu", loss='categorical_crossentropy'):
    model = Sequential()
    model.add(Conv1D(filters=1,
                     kernel_size=1,
                     strides=1,
                     input_shape=(x_train.shape[1],1), 
                     activation=activation))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss=loss, 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    print(model.summary())
    return model

def grid_search():

    model = KerasClassifier(build_fn=create_model, verbose=0)
    # define the grid search parameters
    optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    activation = ["relu", "softmax", "sigmoid","tanh"]
    batch_size = [10, 20, 40, 60, 80, 100]
    epochs = [10, 50, 100]
    param_grid = dict(optimizer=optimizer, epochs=epochs, batch_size=batch_size, activation=activation)
    
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
    grid.fit(x_train, y_train)
    
    print(f"Best estimator:{grid.best_estimator_}\n\nBest params:{grid.best_params_}")
    return grid



model = grid_search()




def eval_model(model):
    _, accuracy = model.evaluate(x_test, y_test, verbose=1)
    print("Model accuracy:",accuracy)


Model: "sequential_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_50 (Conv1D)           (None, 16, 1)             2         
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 8, 1)              0         
_________________________________________________________________
flatten_36 (Flatten)         (None, 8)                 0         
_________________________________________________________________
dense_70 (Dense)             (None, 100)               900       
_________________________________________________________________
dense_71 (Dense)             (None, 3)                 303       
Total params: 1,205
Trainable params: 1,205
Non-trainable params: 0
_________________________________________________________________
None
Best estimator:<keras.wrappers.scikit_learn.KerasClassifier object at 0x7ffb5c0101d0>

Best params:{'activation'

In [148]:
model.best_score_

0.9400471847029727

In [110]:
eval_model(model)

112/112 [==============================] - 0s 788us/step
Model accuracy: 0.875
